In [30]:
from settings import Settings
from utils.datasets import MaestroDataset
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.nn.init
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.optim as optim

In [31]:
settings = Settings() # Set Extra Settings in settings.py

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


In [32]:
dataset = MaestroDataset("train", settings=settings)

In [33]:
print('total number : {}'.format(len(dataset)))

total number : 144910


In [34]:
filt, vel = dataset[10]

In [35]:
filt[:64].shape

(64, 128)

In [36]:
class lstm_Model(nn.Module):
    def __init__(self, n_of_frames,n_of_cnn_frames, kernel_size, output_channel1, output_channel2):
        super(lstm_Model, self).__init__()

        self.n_of_frames = settings.length
        self.n_of_cnn_frames = n_of_cnn_frames
        self.kernel_size = kernel_size
        self.output_channel1 = output_channel1
        self.output_channel2 = output_channel2
        self.Rw1 = n_of_cnn_frames - kernel_size + 1
        self.Rh1 = 128 - kernel_size + 1
        self.Rw2 = (self.Rw1//2) - kernel_size + 1
        self.Rh2 = (self.Rh1//2) - kernel_size + 1
        self.Rw = self.Rw2//2
        self.Rh = self.Rh2//2
        self.sigmoid = nn.Sigmoid()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, output_channel1, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(output_channel1, output_channel2, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.LSTM(1*self.Rh*self.Rw, 128)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out, (_, _) = self.layer3(out.view(x.size(0), 1, -1))
        out = self.sigmoid(out)

        return out

    def loss(self, output, target):
        loss = nn.MSELoss(output.view(1, -1), target)

        return loss

In [44]:
cnt=0
model = lstm_Model(settings.length, settings.n_of_cnn_frames, 5, 1, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss().to(device)

for filt, vel in dataset:
    cnt+=1
    if cnt>=1000: break
        
    for it in range(0,settings.length):
        optimizer.zero_grad()
        frame = filt[it:it+settings.n_of_cnn_frames]
        if len(frame)< settings.n_of_cnn_frames:
            frame=np.append(frame, [np.zeros(128, dtype=bool)]*(settings.n_of_cnn_frames-len(frame)),axis=0)
        out = model(torch.Tensor(frame).view(1,1,settings.n_of_cnn_frames,-1).to(device))
        #print(out)
        loss = criterion(out, torch.Tensor(vel).to(device))
        loss.backward()
        optimizer.step()
        print(cnt, ", ", it, " LOSS: ", loss.item())

1 ,  0  LOSS:  0.24989362061023712
1 ,  1  LOSS:  0.24976156651973724
1 ,  2  LOSS:  0.24962955713272095
1 ,  3  LOSS:  0.24949754774570465
1 ,  4  LOSS:  0.24936553835868835
1 ,  5  LOSS:  0.24923351407051086
1 ,  6  LOSS:  0.24910147488117218
1 ,  7  LOSS:  0.2489694058895111
1 ,  8  LOSS:  0.24883730709552765
1 ,  9  LOSS:  0.2487051635980606
1 ,  10  LOSS:  0.24857297539710999
1 ,  11  LOSS:  0.2484407126903534
1 ,  12  LOSS:  0.24830836057662964
1 ,  13  LOSS:  0.2481759488582611
1 ,  14  LOSS:  0.24804343283176422
1 ,  15  LOSS:  0.24791081249713898
1 ,  16  LOSS:  0.24777807295322418
1 ,  17  LOSS:  0.24764521420001984
1 ,  18  LOSS:  0.24751219153404236
1 ,  19  LOSS:  0.24737901985645294
1 ,  20  LOSS:  0.2472456693649292
1 ,  21  LOSS:  0.24711215496063232
1 ,  22  LOSS:  0.24697841703891754
1 ,  23  LOSS:  0.24684448540210724
1 ,  24  LOSS:  0.24671031534671783
1 ,  25  LOSS:  0.24657589197158813
1 ,  26  LOSS:  0.24644121527671814
1 ,  27  LOSS:  0.24630627036094666
1 ,  28

3 ,  0  LOSS:  0.20109891891479492
3 ,  1  LOSS:  0.20087401568889618
3 ,  2  LOSS:  0.20064933598041534
3 ,  3  LOSS:  0.20042487978935242
3 ,  4  LOSS:  0.2002006471157074
3 ,  5  LOSS:  0.19997666776180267
3 ,  6  LOSS:  0.19975292682647705
3 ,  7  LOSS:  0.19952943921089172
3 ,  8  LOSS:  0.1993062049150467
3 ,  9  LOSS:  0.19908323884010315
3 ,  10  LOSS:  0.1988605409860611
3 ,  11  LOSS:  0.19863811135292053
3 ,  12  LOSS:  0.19841594994068146
3 ,  13  LOSS:  0.19819408655166626
3 ,  14  LOSS:  0.19797250628471375
3 ,  15  LOSS:  0.1977512091398239
3 ,  16  LOSS:  0.19753022491931915
3 ,  17  LOSS:  0.19730953872203827
3 ,  18  LOSS:  0.19708915054798126
3 ,  19  LOSS:  0.19686907529830933
3 ,  20  LOSS:  0.19664931297302246
3 ,  21  LOSS:  0.19642989337444305
3 ,  22  LOSS:  0.19621077179908752
3 ,  23  LOSS:  0.19599197804927826
3 ,  24  LOSS:  0.19577352702617645
3 ,  25  LOSS:  0.1955554038286209
3 ,  26  LOSS:  0.19533762335777283
3 ,  27  LOSS:  0.1951201856136322
3 ,  28 

4 ,  105  LOSS:  0.15980933606624603
4 ,  106  LOSS:  0.15968355536460876
4 ,  107  LOSS:  0.15955816209316254
4 ,  108  LOSS:  0.15943315625190735
4 ,  109  LOSS:  0.159308522939682
4 ,  110  LOSS:  0.1591842621564865
4 ,  111  LOSS:  0.15906038880348206
4 ,  112  LOSS:  0.15893688797950745
4 ,  113  LOSS:  0.15881377458572388
4 ,  114  LOSS:  0.15869101881980896
4 ,  115  LOSS:  0.15856865048408508
4 ,  116  LOSS:  0.15844663977622986
4 ,  117  LOSS:  0.15832501649856567
4 ,  118  LOSS:  0.15820375084877014
4 ,  119  LOSS:  0.15808285772800446
4 ,  120  LOSS:  0.15796233713626862
4 ,  121  LOSS:  0.15784217417240143
4 ,  122  LOSS:  0.1577223837375641
4 ,  123  LOSS:  0.1576029509305954
4 ,  124  LOSS:  0.15748389065265656
4 ,  125  LOSS:  0.15736518800258636
4 ,  126  LOSS:  0.15724684298038483
4 ,  127  LOSS:  0.15712885558605194
5 ,  0  LOSS:  0.1568533480167389
5 ,  1  LOSS:  0.15673625469207764
5 ,  2  LOSS:  0.15661953389644623
5 ,  3  LOSS:  0.15650317072868347
5 ,  4  LOSS:  

6 ,  109  LOSS:  0.136825829744339
6 ,  110  LOSS:  0.13676626980304718
6 ,  111  LOSS:  0.1367068588733673
6 ,  112  LOSS:  0.13664761185646057
6 ,  113  LOSS:  0.13658851385116577
6 ,  114  LOSS:  0.1365295648574829
6 ,  115  LOSS:  0.136470764875412
6 ,  116  LOSS:  0.136412113904953
6 ,  117  LOSS:  0.13635362684726715
6 ,  118  LOSS:  0.13629528880119324
6 ,  119  LOSS:  0.13623708486557007
6 ,  120  LOSS:  0.13617902994155884
6 ,  121  LOSS:  0.13612113893032074
6 ,  122  LOSS:  0.1360633820295334
6 ,  123  LOSS:  0.13600578904151917
6 ,  124  LOSS:  0.1359483301639557
6 ,  125  LOSS:  0.13589102029800415
6 ,  126  LOSS:  0.13583384454250336
6 ,  127  LOSS:  0.1357768326997757
7 ,  0  LOSS:  0.13580754399299622
7 ,  1  LOSS:  0.13575075566768646
7 ,  2  LOSS:  0.13569411635398865
7 ,  3  LOSS:  0.13563761115074158
7 ,  4  LOSS:  0.13558124005794525
7 ,  5  LOSS:  0.13552501797676086
7 ,  6  LOSS:  0.13546893000602722
7 ,  7  LOSS:  0.13541299104690552
7 ,  8  LOSS:  0.13535718619

8 ,  107  LOSS:  0.12554563581943512
8 ,  108  LOSS:  0.12551249563694
8 ,  109  LOSS:  0.12547942996025085
8 ,  110  LOSS:  0.12544642388820648
8 ,  111  LOSS:  0.12541349232196808
8 ,  112  LOSS:  0.12538063526153564
8 ,  113  LOSS:  0.12534783780574799
8 ,  114  LOSS:  0.1253150999546051
8 ,  115  LOSS:  0.12528245151042938
8 ,  116  LOSS:  0.12524986267089844
8 ,  117  LOSS:  0.12521733343601227
8 ,  118  LOSS:  0.12518487870693207
8 ,  119  LOSS:  0.12515248358249664
8 ,  120  LOSS:  0.1251201629638672
8 ,  121  LOSS:  0.1250879168510437
8 ,  122  LOSS:  0.1250557154417038
8 ,  123  LOSS:  0.12502360343933105
8 ,  124  LOSS:  0.12499155104160309
8 ,  125  LOSS:  0.1249595582485199
8 ,  126  LOSS:  0.12492763251066208
8 ,  127  LOSS:  0.12489578127861023
9 ,  0  LOSS:  0.12486964464187622
9 ,  1  LOSS:  0.12483789026737213
9 ,  2  LOSS:  0.12480621039867401
9 ,  3  LOSS:  0.12477459013462067
9 ,  4  LOSS:  0.1247430294752121
9 ,  5  LOSS:  0.1247115358710289
9 ,  6  LOSS:  0.124680

...

159 ,  80  LOSS:  0.10121001303195953
159 ,  81  LOSS:  0.10121000558137894
159 ,  82  LOSS:  0.10121000558137894
159 ,  83  LOSS:  0.10121000558137894
159 ,  84  LOSS:  0.10121000558137894
159 ,  85  LOSS:  0.10121000558137894
159 ,  86  LOSS:  0.10121000558137894
159 ,  87  LOSS:  0.10121000558137894
159 ,  88  LOSS:  0.10121000558137894
159 ,  89  LOSS:  0.10121000558137894
159 ,  90  LOSS:  0.10121000558137894
159 ,  91  LOSS:  0.10121000558137894
159 ,  92  LOSS:  0.10121000558137894
159 ,  93  LOSS:  0.10121000558137894
159 ,  94  LOSS:  0.10121000558137894
159 ,  95  LOSS:  0.10121000558137894
159 ,  96  LOSS:  0.10121000558137894
159 ,  97  LOSS:  0.10121000558137894
159 ,  98  LOSS:  0.10120999813079834
159 ,  99  LOSS:  0.10120999813079834
159 ,  100  LOSS:  0.10120999813079834
159 ,  101  LOSS:  0.10120999813079834
159 ,  102  LOSS:  0.10120999813079834
159 ,  103  LOSS:  0.10120999813079834
159 ,  104  LOSS:  0.10120999813079834
159 ,  105  LOSS:  0.10120999813079834
159 , 

161 ,  48  LOSS:  0.1012403666973114
161 ,  49  LOSS:  0.1012403666973114
161 ,  50  LOSS:  0.1012403666973114
161 ,  51  LOSS:  0.1012403666973114
161 ,  52  LOSS:  0.1012403666973114
161 ,  53  LOSS:  0.1012403666973114
161 ,  54  LOSS:  0.1012403592467308
161 ,  55  LOSS:  0.1012403592467308
161 ,  56  LOSS:  0.1012403592467308
161 ,  57  LOSS:  0.1012403592467308
161 ,  58  LOSS:  0.1012403592467308
161 ,  59  LOSS:  0.1012403592467308
161 ,  60  LOSS:  0.1012403592467308
161 ,  61  LOSS:  0.1012403592467308
161 ,  62  LOSS:  0.1012403592467308
161 ,  63  LOSS:  0.1012403592467308
161 ,  64  LOSS:  0.1012403592467308
161 ,  65  LOSS:  0.1012403592467308
161 ,  66  LOSS:  0.1012403592467308
161 ,  67  LOSS:  0.1012403592467308
161 ,  68  LOSS:  0.1012403592467308
161 ,  69  LOSS:  0.1012403592467308
161 ,  70  LOSS:  0.1012403592467308
161 ,  71  LOSS:  0.1012403592467308
161 ,  72  LOSS:  0.1012403592467308
161 ,  73  LOSS:  0.10124035179615021
161 ,  74  LOSS:  0.10124035179615021

163 ,  15  LOSS:  0.1012810617685318
163 ,  16  LOSS:  0.1012810617685318
163 ,  17  LOSS:  0.1012810617685318
163 ,  18  LOSS:  0.1012810617685318
163 ,  19  LOSS:  0.1012810617685318
163 ,  20  LOSS:  0.1012810617685318
163 ,  21  LOSS:  0.1012810617685318
163 ,  22  LOSS:  0.1012810617685318
163 ,  23  LOSS:  0.1012810617685318
163 ,  24  LOSS:  0.1012810617685318
163 ,  25  LOSS:  0.1012810617685318
163 ,  26  LOSS:  0.1012810617685318
163 ,  27  LOSS:  0.1012810617685318
163 ,  28  LOSS:  0.1012810617685318
163 ,  29  LOSS:  0.1012810617685318
163 ,  30  LOSS:  0.1012810617685318
163 ,  31  LOSS:  0.1012810617685318
163 ,  32  LOSS:  0.1012810617685318
163 ,  33  LOSS:  0.1012810617685318
163 ,  34  LOSS:  0.1012810617685318
163 ,  35  LOSS:  0.1012810617685318
163 ,  36  LOSS:  0.1012810617685318
163 ,  37  LOSS:  0.1012810543179512
163 ,  38  LOSS:  0.1012810543179512
163 ,  39  LOSS:  0.1012810543179512
163 ,  40  LOSS:  0.1012810543179512
163 ,  41  LOSS:  0.1012810543179512
1

164 ,  106  LOSS:  0.10127721726894379
164 ,  107  LOSS:  0.10127721726894379
164 ,  108  LOSS:  0.10127721726894379
164 ,  109  LOSS:  0.10127721726894379
164 ,  110  LOSS:  0.10127721726894379
164 ,  111  LOSS:  0.10127721726894379
164 ,  112  LOSS:  0.10127721726894379
164 ,  113  LOSS:  0.10127721726894379
164 ,  114  LOSS:  0.10127720981836319
164 ,  115  LOSS:  0.10127720981836319
164 ,  116  LOSS:  0.10127720981836319
164 ,  117  LOSS:  0.10127720981836319
164 ,  118  LOSS:  0.10127720981836319
164 ,  119  LOSS:  0.10127720981836319
164 ,  120  LOSS:  0.10127720981836319
164 ,  121  LOSS:  0.10127720981836319
164 ,  122  LOSS:  0.10127720981836319
164 ,  123  LOSS:  0.10127720981836319
164 ,  124  LOSS:  0.10127720981836319
164 ,  125  LOSS:  0.10127720981836319
164 ,  126  LOSS:  0.10127720981836319
164 ,  127  LOSS:  0.10127720981836319
165 ,  0  LOSS:  0.1012892797589302
165 ,  1  LOSS:  0.1012892797589302
165 ,  2  LOSS:  0.1012892797589302
165 ,  3  LOSS:  0.101289279758930

KeyboardInterrupt: 